In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Convert categorical columns

dataset['sex'] = dataset['sex'].map({'male': 0, 'female': 1})
dataset['smoker'] = dataset['smoker'].map({'no': 0, 'yes': 1})


In [ ]:
split = int(len(dataset)) - int(len(dataset)*0.2)
train_dataset = dataset[:split]
test_dataset = dataset[split:]
test_dataset

In [ ]:
region_to_index = {
    'northwest': 0,
    'northeast': 1,
    'southeast': 2,
    'southwest': 3
}
regions = ['northwest', 'southwest', 'southeast', 'northeast', 'northwest']

test_dataset['region'] = test_dataset['region'].map(region_to_index)
train_dataset['region'] = train_dataset['region'].map(region_to_index)


test_dataset

In [ ]:
# Split into features and labels
test_labels = test_dataset['expenses']
train_labels = train_dataset['expenses']

train_dataset = train_dataset.drop('expenses', axis=1)
test_dataset = test_dataset.drop('expenses', axis=1)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=[train_dataset.shape[1]]), # Use Input layer
    layers.Dense(64, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1)
])

model.compile(
    optimizer='adam',
    loss='mae',  # mean absolute error
    metrics=['mae', 'mse']
)

In [ ]:
loss, mae, mse = model.evaluate(test_dataset, test_labels)
print(f'MAE on test set: ${mae:.2f}')


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_data_scaled = scaler.fit_transform(train_dataset)
test_data_scaled = scaler.transform(test_dataset)

train_data_scaled = pd.DataFrame(train_data_scaled, columns=train_dataset.columns)
test_data_scaled = pd.DataFrame(test_data_scaled, columns=test_dataset.columns)

display(train_data_scaled.head())
display(test_data_scaled.head())

In [ ]:
history = model.fit(
    train_data_scaled, train_labels,
    epochs=100,
    validation_split=0.2,
    verbose=1
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
